In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
from datetime import datetime as dt
import cftime

In [2]:
CWatM_DATA_PATH = "../data/raw/CWatM_ISIMIP3_4Thorsten"

area = glob.glob(CWatM_DATA_PATH + "/areamaps/*.nc")

gw = glob.glob(CWatM_DATA_PATH + "/groundwater/*.nc")

lc_forest = glob.glob(CWatM_DATA_PATH + "/landcover/forest/*.nc")#time
lc_grassland = glob.glob(CWatM_DATA_PATH + "/landcover/grassland/*.nc")#time
lc_irrNonPaddy = glob.glob(CWatM_DATA_PATH + "/landcover/irrNonPaddy/*.nc")#time
lc_irrPaddy = glob.glob(CWatM_DATA_PATH + "/landcover/irrPaddy/*.nc")#time

ls = glob.glob(CWatM_DATA_PATH + "/landsurface/*.nc")#time
ls_albedo = glob.glob(CWatM_DATA_PATH + "/landsurface/albedo/*.nc")#time
ls_topo = glob.glob(CWatM_DATA_PATH + "/landsurface/topo/*.nc")
ls_waterDemand = glob.glob(CWatM_DATA_PATH + "/landsurface/waterDemand/*.nc")#times

r = glob.glob(CWatM_DATA_PATH + "/routing/*.nc") # time
r_kinematic = glob.glob(CWatM_DATA_PATH + "/routing/kinematic/*.nc")#time
r_lakesreservoirs = glob.glob(CWatM_DATA_PATH + "/routing/lakesreservoirs/*.nc")#time

soil = glob.glob(CWatM_DATA_PATH + "/soil/*.nc")

In [3]:
sonst = area + gw + soil + ls_topo
lcs = lc_forest + lc_grassland + lc_irrNonPaddy + lc_irrPaddy
ls_oWD = ls + ls_albedo
#ls_waterDemand
rs = r + r_kinematic + r_lakesreservoirs

In [ ]:
#sonst
ds_sonstiges = xr.merge([xr.open_dataset(i) for i in sonst])
ds_sonstiges

In [ ]:
#lcs
ds_lc_forest = xr.merge([xr.open_dataset(i) for i in lc_forest])
df_lc_forest = ds_lc_forest.to_dataframe()
df_lc_forest.rename(columns=lambda x: x + '_forest', inplace=True)
ds_lc_grassland = xr.merge([xr.open_dataset(i) for i in lc_grassland])
df_lc_grassland = ds_lc_forest.to_dataframe()
df_lc_grassland.rename(columns=lambda x: x + '_grassland', inplace=True)
ds_lc_irrNonPaddy = xr.merge([xr.open_dataset(i) for i in lc_irrNonPaddy])
df_lc_irrNonPaddy = ds_lc_forest.to_dataframe()
df_lc_irrNonPaddy.rename(columns=lambda x: x + '_irrNonPaddy', inplace=True)
ds_lc_irrPaddy = xr.merge([xr.open_dataset(i) for i in lc_irrPaddy])
df_lc_irrPaddy = ds_lc_forest.to_dataframe()
df_lc_irrPaddy.rename(columns=lambda x: x + '_irrPaddy', inplace=True)
df_lc = pd.concat([df_lc_forest, df_lc_grassland, df_lc_irrNonPaddy, df_lc_irrPaddy], axis=1)
ds_lc = df_lc.to_xarray()
ds_lc

In [ ]:
ds_lc_resampled = ds_lc.resample(time="1Y").sum()
ds_lc_resampled_mean = ds_lc_resampled.mean("time")

In [ ]:
#ls_oWD
ds_ls_oWD =  xr.merge([xr.open_dataset(i) for i in ls_oWD])
ds_ls_oWD

In [ ]:
ds_ls_oWD_resampled = ds_ls_oWD.resample(time="1Y").sum()
ds_ls_oWD_timeslice = ds_ls_oWD_resampled.sel(time=slice(dt(1975, 1, 1), dt(2004, 12, 31)))
ds_ls_oWD_mean = ds_ls_oWD_timeslice.mean("time")

In [ ]:
sorted(ls_waterDemand)

In [ ]:
#ls_waterDemand
ds_ls_wD_times = xr.merge([xr.open_dataset(sorted(ls_waterDemand)[i], decode_times=False) for i in [0, 2, 4]])

time = ds_ls_oWD.get_index('time')
t1 = time[0:151]
t2 = time[162:]
time_yearly = np.concatenate((t1, t2))

df_ls_wD_times = ds_ls_wD_times.to_dataframe()
idx = df_ls_wD_times.index
nidx = idx.set_levels(time_yearly, level = 'time')
df_ls_wD_times.index = nidx
ds_ls_wD_times_corr = df_ls_wD_times.to_xarray()
ds_ls_wD_times_corr

In [ ]:
ds_ls_wD_times_corr_resampled = ds_ls_wD_times_corr.resample(time="1Y").sum()
ds_ls_wD_times_corr_timeslice = ds_ls_wD_times_corr_resampled.sel(time=slice(dt(1975, 1, 1), dt(2004, 12, 31)))
ds_ls_wD_times_corr_mean = ds_ls_wD_times_corr_timeslice.mean("time")

In [ ]:
sorted(ls_waterDemand)

In [ ]:
ds_ls_wD_times2 = xr.merge([xr.open_dataset(sorted(ls_waterDemand)[i]) for i in [1, 3]])
ds_ls_wD_times2

In [ ]:
ds_ls_wD_times2_resampled = ds_ls_wD_times2.resample(time="1Y").sum()
ds_ls_wD_times2_timeslice = ds_ls_wD_times2_resampled.sel(time=slice(dt(1975, 1, 1), dt(2004, 12, 31)))
ds_ls_wD_times2_mean = ds_ls_wD_times2_timeslice.mean("time")

In [ ]:
sorted(rs)

In [ ]:
#rs
ds_routing0 = xr.open_dataset(sorted(rs)[0])
df_routing0 = ds_routing0.to_dataframe()
ds_routing0 = df_routing0.to_xarray()
ds_routing_notimes1 = xr.merge([xr.open_dataset(i) for i in sorted(rs)[1:13]])
ds_routing_notimes2 = xr.merge([xr.open_dataset(i) for i in sorted(rs)[14:]])
ds_routing_notimes = xr.merge([ds_routing0, ds_routing_notimes1, ds_routing_notimes2])
ds_routing_notimes

In [ ]:
ds_routing_times = xr.open_dataset(sorted(rs)[13])
ds_routing_times

In [ ]:
ds_routing_times_resampled = ds_routing_times.resample(time="1Y").sum()
ds_routing_times_timeslice = ds_routing_times_resampled.sel(time=slice(dt(1975, 1, 1), dt(2004, 12, 31)))
ds_routing_times_mean = ds_routing_times_timeslice.mean("time")

In [ ]:
#all variables without time
ds_all = xr.merge([ds_sonstiges, ds_lc_resampled_mean, ds_ls_oWD_mean, ds_ls_wD_times_corr_mean, ds_ls_wD_times2_mean, ds_routing_notimes, ds_routing_times_mean])
ds_all

In [21]:
ds_all.to_netcdf("../data/processed/CWatM_data/all.nc")